In [1]:
import pandas as pd

In [2]:
data=pd.read_csv('data/Credit_Card.csv')

In [3]:
data.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,payment delay for two months,payment delay for two months,pay duly,pay duly,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,pay duly,payment delay for two months,payment on time,payment on time,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,payment on time,payment on time,payment on time,payment on time,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,payment on time,payment on time,payment on time,payment on time,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,pay duly,payment on time,pay duly,payment on time,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [53]:
data.to_csv('data/Credit_Card.csv',index=False)

In [4]:
target_column = "default.payment.next.month"
drop_columns = [
                "ID", "AGE", "SEX", "EDUCATION", "MARRIAGE",
                "BILL_AMT1", "BILL_AMT2", "BILL_AMT3", "BILL_AMT4", "BILL_AMT5", "BILL_AMT6",
                "PAY_AMT1", "PAY_AMT2", "PAY_AMT3", "PAY_AMT4", "PAY_AMT5", "PAY_AMT6"
                 
        ]

In [5]:
X=data.drop(drop_columns,axis=1)

In [6]:
X.head(1)

,LIMIT_BAL,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,default.payment.next.month
0,20000.0,payment delay for two months,payment delay for two months,pay duly,pay duly,no consumption,no consumption,1


In [39]:
y=data[target_column]

In [40]:
num_features = ["LIMIT_BAL"]
cat_features = ['PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']

        
pay_categories = [[
                "no consumption", "pay duly", "payment on time", "payment delay for one month",
                "payment delay for two months", "payment delay for three months",
                "payment delay for four months", "payment delay for five months",
                "payment delay for six months", "payment delay for seven months",
                "payment delay for eight months"
             ] ] * len(cat_features)

In [41]:
from sklearn.preprocessing import OrdinalEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [42]:
num_pipeline = Pipeline([
                ("imputer", SimpleImputer(strategy="median")),  # Fix order
                ("scaler", StandardScaler())
            ])
           

        # Define categorical pipeline
cat_pipeline = Pipeline([
                ("encoder", OrdinalEncoder(categories=pay_categories))
            ])
            

        # Combine both pipelines using ColumnTransformer
preprocessor = ColumnTransformer([
                ("num_pipeline", num_pipeline, num_features),
                ("cat_pipeline", cat_pipeline, cat_features)
            ])
            

In [43]:
from sklearn.model_selection import train_test_split

In [44]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=2)

In [45]:
X_train

,LIMIT_BAL,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6
19282,230000.0,pay duly,pay duly,pay duly,pay duly,pay duly,pay duly
1713,390000.0,payment on time,payment on time,payment on time,payment on time,payment on time,payment on time
15937,50000.0,payment delay for one month,payment delay for two months,payment delay for two months,payment delay for two months,payment delay for two months,payment delay for three months
699,300000.0,payment on time,payment on time,payment on time,payment on time,payment delay for two months,pay duly
29598,70000.0,payment on time,payment on time,payment on time,payment on time,payment on time,payment on time
...,...,...,...,...,...,...,...
18898,140000.0,payment on time,payment on time,pay duly,payment on time,payment on time,payment on time
11798,50000.0,payment on time,payment on time,payment on time,payment on time,payment on time,payment on time
6637,610000.0,payment on time,payment on time,payment on time,payment on time,payment on time,payment on time
2575,180000.0,payment on time,payment on time,payment on time,payment on time,payment on time,payment on time


In [46]:
X_scaled=preprocessor.fit_transform(X_train)